In [10]:
import os
import pandas as pd
import numpy as np

from scipy import signal, fftpack
from scipy.fft import fft, fftfreq, fftshift

In [11]:
# Define functions to calculate peak values

def calculate_highest_peak_raw(signal):
    return np.max(signal)

def calculate_highest_filtered(signal):
    return np.max(signal)

def calculate_highest_peak_rectified(signal):
    return np.max(signal)

def calculate_highest_fft(signal):
    return np.max(signal)

def calculate_highest_peak_psd(signal):
    return np.max(signal)

# Folder path containing the CSV files
folder_path = "E:/Pengolahan Data CD Bone-USG/Data Revisi 14082023/CSV/All/"

# List of file names in the folder
file_names = os.listdir(folder_path)

# Filter only CSV files
file_paths = [os.path.join(folder_path, file_name) for file_name in file_names if file_name.endswith(".csv")]

# Initialize an empty list to store peak values
peak_results = []

In [12]:
for file_path in file_paths:
    df_uncut = pd.read_csv(file_path)
    start_index = 1
    end_index = 1551
    df = df_uncut[start_index:end_index]
    data = df['Y'].values

    # ... (rest of your processing code)
    lowcut = 2.0
    highcut = 10.0
    fs = 128.0
    order = 7

    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')

    # Filter bandpass
    filtered_signal = signal.filtfilt(b, a, data)

    # Rectify Signal
    rectified_signal = np.abs(filtered_signal)

    # One-sided FFT
    signal_fft = np.fft.fft(filtered_signal)
    signal_freq = np.fft.fftfreq(len(filtered_signal), 1/fs)
    n = len(filtered_signal)
    freq = signal_freq[0:int(n/2)]
    fft_amp = np.abs(signal_fft)[0:int(n/2)]/n*2

    # One-sided PSD
    psd = (1/(fs*n)) * np.square(np.abs(signal_fft[:n//2]))
    freq_psd = signal_freq[0:int(n/2)]
    psd_one_side = psd[0:int(n/2)]
    
    # # Calculate highest peaks
    # highest_peak_raw = calculate_highest_peak_raw(data)
    # highest_peak_rectified = calculate_highest_peak_rectified(rectified_signal)
    # highest_peak_filtered = calculate_highest_filtered(filtered_signal)
    # highest_peak_fft = calculate_highest_fft(fft_amp)
    # highest_peak_psd = calculate_highest_peak_psd(psd_one_side)

    # Calculate highest peaks
    # highest_peak_raw = round(calculate_highest_peak_raw(data), 6)
    # highest_peak_rectified = round(calculate_highest_peak_rectified(rectified_signal), 6)
    # highest_peak_filtered = round(calculate_highest_filtered(filtered_signal), 6)
    # highest_peak_fft = round(calculate_highest_fft(fft_amp), 6)
    # highest_peak_psd = round(calculate_highest_peak_psd(psd_one_side), 6)

    # Calculate highest peaks with decimal precision
    highest_peak_raw = '{:.6f}'.format(calculate_highest_peak_psd(data))
    highest_peak_rectified = '{:.6f}'.format(calculate_highest_peak_rectified(rectified_signal))
    highest_peak_filtered = '{:.6f}'.format(calculate_highest_filtered(filtered_signal))
    highest_peak_fft = '{:.6f}'.format(calculate_highest_fft(fft_amp))
    highest_peak_psd = '{:.6f}'.format(calculate_highest_peak_psd(psd_one_side))



    # Append results to the list
    peak_results.append({
        "File": os.path.basename(file_path),
        "Highest_Peak_Raw": highest_peak_raw,
        "Highest_Peak_Filtered": highest_peak_filtered,
        "Highest_Peak_Rectified": highest_peak_rectified,
        "Highest_Peak_FFT": highest_peak_fft,
        "Highest_Peak_PSD": highest_peak_psd
    })

# Create a DataFrame from the list of results
peak_results_df = pd.DataFrame(peak_results)

# Save the DataFrame to a new CSV file
output_file_path = os.path.join(folder_path, "Hasil_All_14082023_V4.csv")
peak_results_df.to_csv(output_file_path, index=False)
